# Data Engineer
## Introduction
The following tasks are to test the basic skillset of a data engineer.
You aren’t expected to spend more than 30-60 minutes on this task
You are free to use the internet but must solve this task yourself.


* You will find three csv files attached whih you will need to use to answer the below questions: 1.goalscorers.csv, results.csv & shootouts.csv
* Add your answers to this file:answers

## Objectives
1. Create a query that calculates the average number of goals per game between 1900 and 2000.
2. Create a query that counts the number of shootouts wins by country and arrange in alphabetical order.
3. Create a reliable key that allows the joining together of goal scorers, results, and shootouts.
4. Create a query that identifies which teams have won a penalty shootout after a 1-1 draw.
5. Create a query that identifies the top goal scorer by tournament, and what percentage that equates to for all goals scored in the tournament.

Additional (If you have time)
1. Create and additional column that flags records with data quality issues
2. Resolve the identified quality issues

In [63]:
import pandas as pd

goalscorers = pd.read_csv('goalscorers.csv')
results = pd.read_csv('results.csv')
shootouts = pd.read_csv('shootouts.csv')

In [64]:
print(goalscorers.head())

         date  home_team away_team       team            scorer  minute  \
0  1916-07-02      Chile   Uruguay    Uruguay   José Piendibene    44.0   
1  1916-07-02      Chile   Uruguay    Uruguay  Isabelino Gradín    55.0   
2  1916-07-02      Chile   Uruguay    Uruguay  Isabelino Gradín    70.0   
3  1916-07-02      Chile   Uruguay    Uruguay   José Piendibene    75.0   
4  1916-07-06  Argentina     Chile  Argentina     Alberto Ohaco     2.0   

  own_goal penalty  
0    False   False  
1    False   False  
2    False   False  
3    False   False  
4    False   False  


In [65]:
print(results.head())

         date home_team away_team  home_score  away_score tournament     city  \
0  1872-11-30  Scotland   England           0           0   Friendly  Glasgow   
1  1873-03-08   England  Scotland           4           2   Friendly   London   
2  1874-03-07  Scotland   England           2           1   Friendly  Glasgow   
3  1875-03-06   England  Scotland           2           2   Friendly   London   
4  1876-03-04  Scotland   England           3           0   Friendly  Glasgow   

    country  neutral  
0  Scotland    False  
1   England    False  
2  Scotland    False  
3   England    False  
4  Scotland    False  


In [66]:
print(shootouts.head())

         date    home_team         away_team       winner first_shooter
0  1967-08-22        India            Taiwan       Taiwan           NaN
1  1971-11-14  South Korea  Vietnam Republic  South Korea           NaN
2  1972-05-07  South Korea              Iraq         Iraq           NaN
3  1972-05-17     Thailand       South Korea  South Korea           NaN
4  1972-05-19     Thailand          Cambodia     Thailand           NaN


Task 1: Average Number of Goals per Game (1900-2000)

Filter results for matches between 1900 and 2000, compute the total goals (home_score + away_score) for each match, and calculate the average.

1. Create a query that calculates the average number of goals per game between 1900 and 2000.

In [67]:
# Convert the 'date' column to datetime and extract the year
results['year'] = pd.to_datetime(results['date']).dt.year

# Filter the results for games between 1900 and 2000, and create a copy
filtered_results = results[(results['year'] >= 1900) & (results['year'] <= 2000)].copy()

# Add a new column for total goals
filtered_results['total_goals'] = filtered_results['home_score'] + filtered_results['away_score']

# Calculate the average number of goals per game
average_goals = filtered_results['total_goals'].mean()

# Print the result
print("Average goals per game (1900-2000):", average_goals)

Average goals per game (1900-2000): 3.0704284750337383


Task 2: Shootout Wins by Country

Group shootouts by winner and count the occurrences. Sort alphabetically by country.

2. Create a query that counts the number of shootouts wins by country and arrange in alphabetical order.

In [75]:
shootout_wins = shootouts['winner'].value_counts().reset_index()
shootout_wins.columns = ['country', 'wins']
shootout_wins_sorted = shootout_wins.sort_values(by='country')
print(shootout_wins_sorted)

                 country  wins
79              Abkhazia     2
26               Algeria     7
16                Angola     7
92   Antigua and Barbuda     2
0              Argentina    14
..                   ...   ...
1                 Zambia    13
108             Zanzibar     2
22              Zimbabwe     7
162                Åland     1
104        Åland Islands     2

[163 rows x 2 columns]


Task 3: Reliable Key for Joining

A reliable join key should uniquely identify matches across all datasets. Since all datasets share date, home_team, and away_team, we can use these fields to create a composite key.

3. Create a reliable key that allows the joining together of goal scorers, results, and shootouts.

In [ ]:

# Create composite join keys for each dataset
results['join_key'] = results['date'] + '_' + results['home_team'] + '_' + results['away_team']
goalscorers['join_key'] = goalscorers['date'] + '_' + goalscorers['home_team'] + '_' + goalscorers['away_team']
shootouts['join_key'] = shootouts['date'] + '_' + shootouts['home_team'] + '_' + shootouts['away_team']

# Filter draws (matches where home_score equals away_score)
draws = results[results['home_score'] == results['away_score']].copy()

# Create join_key for the draws dataset using .loc
draws.loc[:, 'join_key'] = draws['date'] + '_' + draws['home_team'] + '_' + draws['away_team']

# Rename overlapping columns in shootouts to avoid conflicts
shootouts_renamed = shootouts.rename(columns={
    'date': 'shootout_date',
    'home_team': 'shootout_home_team',
    'away_team': 'shootout_away_team',
    'winner': 'shootout_winner'
})

# Perform the merge based on 'join_key'
draws_with_shootouts = draws.merge(shootouts_renamed, on='join_key', how='inner')

# Select only the necessary columns
required_columns = ['shootout_date', 'shootout_home_team', 'shootout_away_team', 'shootout_winner']

# Verify if the necessary columns are present
if all(col in draws_with_shootouts.columns for col in required_columns):
    winners = draws_with_shootouts[required_columns]
    print(winners)
else:
    print("Required columns are missing after the merge.")







    shootout_date shootout_home_team shootout_away_team shootout_winner
0      1967-08-22              India             Taiwan          Taiwan
1      1971-11-14        South Korea   Vietnam Republic     South Korea
2      1972-05-07        South Korea               Iraq            Iraq
3      1972-05-17           Thailand        South Korea     South Korea
4      1972-05-19           Thailand           Cambodia        Thailand
..            ...                ...                ...             ...
518    2023-07-12      United States             Panama          Panama
519    2023-07-13              Åland   Falkland Islands           Åland
520    2023-09-07               Iraq              India            Iraq
521    2023-09-10           Thailand               Iraq            Iraq
522    2023-10-13               Iraq              Qatar           Qatar

[523 rows x 4 columns]
Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'ne

In [76]:
print(draws_with_shootouts.columns)

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral', 'year', 'join_key',
       'shootout_date', 'shootout_home_team', 'shootout_away_team',
       'shootout_winner', 'first_shooter'],
      dtype='object')


Task 4: Teams Winning Shootouts After a 1-1 Draw

Filter results for matches with a 1-1 draw, and then cross-reference with shootouts to identify winners.

4. Create a query that identifies which teams have won a penalty shootout after a 1-1 draw.

In [69]:
# Step 1: Filter results for matches that ended in a 1-1 draw
draws_1_1 = results[results['home_score'] == 1 & (results['away_score'] == 1)].copy()

# Step 2: Merge with shootouts based on the composite join key
draws_with_shootouts = draws_1_1.merge(shootouts_renamed, on='join_key', how='inner')

# Step 3: Extract the relevant columns for winners
winners = draws_with_shootouts[['shootout_date', 'shootout_home_team', 'shootout_away_team', 'shootout_winner']].copy()

# Step 4: Rename columns for clarity (optional)
winners.rename(columns={
    'shootout_date': 'date',
    'shootout_home_team': 'home_team',
    'shootout_away_team': 'away_team'
}, inplace=True)

# Step 5: Display the winners
print(winners)

           date      home_team         away_team shootout_winner
0    1967-08-22          India            Taiwan          Taiwan
1    1971-11-14    South Korea  Vietnam Republic     South Korea
2    1972-05-07    South Korea              Iraq            Iraq
3    1972-05-17       Thailand       South Korea     South Korea
4    1973-06-14         Guinea              Mali          Guinea
..          ...            ...               ...             ...
415  2023-07-01          India           Lebanon           India
416  2023-07-04          India            Kuwait           India
417  2023-07-12  United States            Panama          Panama
418  2023-07-13          Åland  Falkland Islands           Åland
419  2023-10-13           Iraq             Qatar           Qatar

[420 rows x 4 columns]


Task 5: Top Goal Scorer by Tournament

Group by tournament and scorer to find the player with the most goals in each tournament. Then calculate their percentage of total tournament goals.

5. Create a query that identifies the top goal scorer by tournament, and what percentage that equates to for all goals scored in the tournament.

In [70]:
# Merge goalscorers with results to add tournament details using the join_key
goalscorers_with_tournament = pd.merge(goalscorers, results[['date', 'home_team', 'away_team', 'tournament', 'join_key']],
                                       on=['join_key'], how='left')

# Total goals by tournament
tournament_goals = goalscorers_with_tournament.groupby('tournament').size().reset_index(name='total_goals')

# Goals by scorer per tournament
scorer_goals = goalscorers_with_tournament.groupby(['tournament', 'scorer']).size().reset_index(name='scorer_goals')

# Find the top scorer per tournament (the one with the most goals)
top_scorer_per_tournament = scorer_goals.loc[scorer_goals.groupby('tournament')['scorer_goals'].idxmax()]

# Merge to calculate the percentage of total goals scored by the top scorer
top_scorer_with_percentage = top_scorer_per_tournament.merge(tournament_goals, on='tournament')
top_scorer_with_percentage['percentage'] = (top_scorer_with_percentage['scorer_goals'] / 
                                            top_scorer_with_percentage['total_goals']) * 100

# Print the result
print(top_scorer_with_percentage[['tournament', 'scorer', 'scorer_goals', 'total_goals', 'percentage']])

                        tournament                   scorer  scorer_goals  \
0                    AFC Asian Cup                 Ali Daei            14   
1           African Cup of Nations             Samuel Eto'o            18   
2                       Baltic Cup          Ēriks Pētersons             9   
3        British Home Championship              Geoff Hurst             4   
4   CONMEBOL–UEFA Cup of Champions         Claudio Caniggia             1   
5               Confederations Cup        Cuauhtémoc Blanco             9   
6                     Copa América  Norberto Doroteo Méndez            17   
7                   FIFA World Cup           Miroslav Klose            16   
8     FIFA World Cup qualification              Carlos Ruiz            39   
9                         Gold Cup           Landon Donovan            18   
10               NAFC Championship        Luis de la Fuente             3   
11             Oceania Nations Cup              Damian Mori            14   

TASK 

Additional

1. Create and additional column that flags records with data quality issues

2. Resolve the identified quality issues


ADDITIONAL 
Task Summary: Flagging and Resolving Data Quality Issues

The task involves creating an additional column in each dataset to flag records with potential data quality issues. 

This column helps identify rows with common issues such as missing values, duplicates, or invalid data. Here’s a summary of the steps taken:

1.	Data Quality Flagging:
•	Missing Values: Rows with missing values in critical columns like date, home_team, and away_team are flagged.
•	Duplicates: Rows with duplicate entries based on the combination of date, home_team, and away_team are flagged.
•	Invalid Data:
•	In the goalscorers dataset, the minute column is checked to ensure valid values (0 to 90).
•	In the results dataset, negative scores for home_score or away_score are flagged as invalid.
•	In the shootouts dataset, rows with missing winner values are flagged.

2.	Implementation:
•	A function flag_data_quality was created to flag issues across each dataset.
•	The flagged issues are stored in a new column, data_quality_flag, which is True if an issue is identified.

3.	Output:
•	The datasets (goalscorers, results, shootouts) are updated with the data_quality_flag column.
•	The rows with flagged issues are printed for inspection and resolution.

In [71]:
# Function to flag data quality issues
def flag_data_quality(df, dataset_name):
    # Create an empty 'data_quality_flag' column initialized to False
    df['data_quality_flag'] = False

    # Flag rows with missing values in critical columns (example columns: 'date', 'home_team', 'away_team')
    if 'date' in df.columns:
        df['data_quality_flag'] = df['data_quality_flag'] | df['date'].isnull()
    if 'home_team' in df.columns:
        df['data_quality_flag'] = df['data_quality_flag'] | df['home_team'].isnull()
    if 'away_team' in df.columns:
        df['data_quality_flag'] = df['data_quality_flag'] | df['away_team'].isnull()
    
    # Flag rows with duplicates based on 'date', 'home_team', and 'away_team'
    df['data_quality_flag'] = df['data_quality_flag'] | df.duplicated(subset=['date', 'home_team', 'away_team'], keep=False)

    # Example of flagging rows with invalid data (can adjust per dataset)
    if dataset_name == "goalscorers" and 'minute' in df.columns:
        df['data_quality_flag'] = df['data_quality_flag'] | ~df['minute'].between(0, 90)  # Flag invalid minutes
    
    if dataset_name == "results":
        # Flag rows with impossible score values (e.g., negative scores)
        df['data_quality_flag'] = df['data_quality_flag'] | (df['home_score'] < 0) | (df['away_score'] < 0)
    
    if dataset_name == "shootouts" and 'winner' in df.columns:
        # Flag rows with missing winners
        df['data_quality_flag'] = df['data_quality_flag'] | df['winner'].isnull()

    return df

# Flag data quality issues for each dataset
goalscorers = flag_data_quality(goalscorers, "goalscorers")
results = flag_data_quality(results, "results")
shootouts = flag_data_quality(shootouts, "shootouts")

In [72]:
# Example: Print rows with data quality issues for each dataset
print("Goalscorers with Data Quality Issues:")
print(goalscorers[goalscorers['data_quality_flag'] == True])

Goalscorers with Data Quality Issues:
             date  home_team away_team       team            scorer  minute  \
0      1916-07-02      Chile   Uruguay    Uruguay   José Piendibene    44.0   
1      1916-07-02      Chile   Uruguay    Uruguay  Isabelino Gradín    55.0   
2      1916-07-02      Chile   Uruguay    Uruguay  Isabelino Gradín    70.0   
3      1916-07-02      Chile   Uruguay    Uruguay   José Piendibene    75.0   
4      1916-07-06  Argentina     Chile  Argentina     Alberto Ohaco     2.0   
...           ...        ...       ...        ...               ...     ...   
43182  2023-11-21     Greece    France     Greece   Tasos Bakasetas    56.0   
43183  2023-11-21     Greece    France     Greece   Fotis Ioannidis    61.0   
43184  2023-11-21     Greece    France     France   Youssouf Fofana    74.0   
43187  2023-11-21      Wales    Turkey      Wales     Neco Williams     7.0   
43188  2023-11-21      Wales    Turkey     Turkey      Yusuf Yazıcı    70.0   

      own_goa

In [73]:
print("\nResults with Data Quality Issues:")
print(results[results['data_quality_flag'] == True])


Results with Data Quality Issues:
             date home_team      away_team  home_score  away_score tournament  \
9072   1974-02-17    Tahiti  New Caledonia           2           1   Friendly   
9073   1974-02-17    Tahiti  New Caledonia           1           2   Friendly   
10554  1977-10-22    Guyana       Barbados           2           0   Friendly   
10555  1977-10-22    Guyana       Barbados           0           0   Friendly   

             city           country  neutral  year  \
9072      Papeete  French Polynesia    False  1974   
9073      Papeete  French Polynesia    False  1974   
10554      Linden            Guyana    False  1977   
10555  Georgetown            Guyana    False  1977   

                              join_key  data_quality_flag  
9072   1974-02-17_Tahiti_New Caledonia               True  
9073   1974-02-17_Tahiti_New Caledonia               True  
10554       1977-10-22_Guyana_Barbados               True  
10555       1977-10-22_Guyana_Barbados          

In [74]:

print("\nShootouts with Data Quality Issues:")
print(shootouts[shootouts['data_quality_flag'] == True])


Shootouts with Data Quality Issues:
Empty DataFrame
Columns: [date, home_team, away_team, winner, first_shooter, join_key, data_quality_flag]
Index: []
